In [1]:
from Bio import Entrez, SeqIO
Entrez.email = "swarajmthorat@gmail.com"  
Entrez.api_key = "9a87d832c19a96cf84d804b411eec0386b08"  # Replace with your API key if you have one

In [2]:
def fetch_surf1_sequences():
    # Search specifically for the SURF1 gene across different species
    search_term = "SURF1 cytochrome c oxidase assembly factor AND (Homo sapiens[Organism] OR Bos taurus[Organism] OR Canis lupus[Organism])"
    
    # Search NCBI nucleotide database for SURF1 sequences
    with Entrez.esearch(db="nucleotide", term=search_term, retmax=5) as handle:
        search_results = Entrez.read(handle)
        ids = search_results["IdList"]
    
    # Fetch sequences in FASTA format
    with Entrez.efetch(db="nucleotide", id=ids, rettype="fasta", retmode="text") as handle:
        sequences = list(SeqIO.parse(handle, "fasta"))
    
    # Save sequences to a FASTA file
    SeqIO.write(sequences, "SURF1_sequences.fasta", "fasta")
    print(f"Downloaded and saved {len(sequences)} SURF1 sequences to 'SURF1_sequences.fasta'")
    
    return sequences

# Run the function to fetch SURF1 sequences
surf1_sequences = fetch_surf1_sequences()

Downloaded and saved 5 SURF1 sequences to 'SURF1_sequences.fasta'


In [3]:
# Initialize a list to store sequence details
parsed_sequences = []

# Parse the SURF1 FASTA file
for record in SeqIO.parse("SURF1_sequences.fasta", "fasta"):
    seq_id = record.id
    description = record.description
    sequence = record.seq
    seq_length = len(sequence)
    
    # Display basic information
    print(f"ID: {seq_id}")
    print(f"Description: {description}")
    print(f"Length: {seq_length}")
    print(f"Sequence: {sequence[:50]}...")  # Display first 50 nucleotides for brevity
    print("-" * 40)
    
    # Append to parsed sequences list
    parsed_sequences.append({
        "ID": seq_id,
        "Description": description,
        "Sequence": str(sequence),
        "Length": seq_length
    })

# Display the total number of sequences parsed
print(f"Total Sequences Parsed: {len(parsed_sequences)}")

ID: NG_008477.2
Description: NG_008477.2 Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), RefSeqGene on chromosome 9
Length: 11730
Sequence: AATAAATAATGGCGTTTGTTGTATGCAGTGTGATCCTATAGCGTTGCCCA...
----------------------------------------
ID: NM_003172.4
Description: NM_003172.4 Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), transcript variant 1, mRNA
Length: 1092
Sequence: CTGCGTCCCGGAAGCGCCCGCGGGGCCGGGTGCGATGGCGGCGGTGGCTG...
----------------------------------------
ID: NM_001280787.1
Description: NM_001280787.1 Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), transcript variant 2, mRNA
Length: 994
Sequence: GCGTCCCGGAAGCGCCCGCGGGGCCGGGTGCGATGGCGGCGGTGGCTGCG...
----------------------------------------
ID: NM_016565.3
Description: NM_016565.3 Homo sapiens cytochrome c oxidase assembly factor 4 homolog (COA4), mRNA; nuclear gene for mitochondrial product
Length: 818
Sequence: GAAAGGATGTACAGAGGATCCCCAACCGCCTGCGAAACCCAAGCCGCCGC...
-----------

In [ ]:

from Bio.Blast import NCBIWWW
from Bio import SeqIO

TypeError: qblast() got an unexpected keyword argument 'api_key'

In [9]:
import time
from Bio.Blast import NCBIWWW
from Bio import SeqIO

# Open the FASTA file and perform BLAST for each sequence
with open("SURF1_sequences.fasta") as fasta_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        print(f"Running BLAST for sequence ID: {record.id}")

        # Perform the BLAST search with a delay
        try:
            result_handle = NCBIWWW.qblast("blastn", "nt", record.seq, hitlist_size=5)
            
            # Save the BLAST output to an XML file for this sequence
            with open(f"{record.id}_blast.xml", "w") as output_handle:
                output_handle.write(result_handle.read())
                
            # Close the result handle after saving
            result_handle.close()
            
            print(f"BLAST for {record.id} completed successfully.")
            
            # Add a delay to avoid rate limiting
            time.sleep(5)  # Wait for 5 seconds between requests
        except Exception as e:
            print(f"An error occurred with sequence {record.id}: {e}")

Running BLAST for sequence ID: NG_008477.2
BLAST for NG_008477.2 completed successfully.
Running BLAST for sequence ID: NM_003172.4
BLAST for NM_003172.4 completed successfully.
Running BLAST for sequence ID: NM_001280787.1
BLAST for NM_001280787.1 completed successfully.
Running BLAST for sequence ID: NM_016565.3
BLAST for NM_016565.3 completed successfully.
Running BLAST for sequence ID: NC_060933.1
An error occurred with sequence NC_060933.1: No RID and no RTOE found in the 'please wait' page, there was probably an error in your request but we could not extract a helpful error message.


NC_060933.1 is a very long sequence, will try submitting only a part of it (e.g., the first 1000 or 2000 nucleotides) 

In [10]:
# Modify this part for the problematic sequence
if record.id == "NC_060933.1":
    # Trim the sequence to the first 2000 bases
    query_sequence = record.seq[:2000]
else:
    query_sequence = record.seq

# Run the BLAST search with the adjusted sequence
try:
    result_handle = NCBIWWW.qblast("blastn", "nt", query_sequence, hitlist_size=5)
    with open(f"{record.id}_blast.xml", "w") as output_handle:
        output_handle.write(result_handle.read())
    result_handle.close()
    print(f"BLAST for {record.id} completed successfully.")
    time.sleep(5)  # Add delay to avoid rate limiting
except Exception as e:
    print(f"An error occurred with sequence {record.id}: {e}")

BLAST for NC_060933.1 completed successfully.


## Lets analyze the blast results

In [11]:
from Bio.Blast import NCBIXML

# List to store parsed results for easy viewing or saving
parsed_results = []

# Loop through each BLAST result file and parse the results
for record in SeqIO.parse("SURF1_sequences.fasta", "fasta"):
    blast_file = f"{record.id}_blast.xml"
    try:
        with open(blast_file) as result_handle:
            blast_record = NCBIXML.read(result_handle)

            # Extract and print the top hits for each sequence
            print(f"\nTop hits for sequence ID: {record.id}")
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    # Filter by an E-value threshold (e.g., less than 1e-10 for high confidence)
                    if hsp.expect < 1e-10:
                        # Collect relevant information
                        hit_info = {
                            "Query ID": record.id,
                            "Hit ID": alignment.hit_id,
                            "Description": alignment.hit_def,
                            "E-value": hsp.expect,
                            "Score": hsp.score,
                            "Query Length": len(record.seq),
                            "Alignment Length": hsp.align_length,
                            "Identity": hsp.identities,
                        }
                        # Print the result
                        print(f"Hit ID: {alignment.hit_id}")
                        print(f"Description: {alignment.hit_def}")
                        print(f"E-value: {hsp.expect}")
                        print(f"Score: {hsp.score}")
                        print("-" * 40)

                        # Append to parsed_results for later use
                        parsed_results.append(hit_info)
    except FileNotFoundError:
        print(f"No BLAST result found for sequence ID: {record.id}")


Top hits for sequence ID: NG_008477.2
Hit ID: gi|2296125236|ref|NG_008477.2|
Description: Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), RefSeqGene on chromosome 9
E-value: 0.0
Score: 23460.0
----------------------------------------
Hit ID: gi|2296125236|ref|NG_008477.2|
Description: Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), RefSeqGene on chromosome 9
E-value: 2.19268e-13
Score: 105.0
----------------------------------------
Hit ID: gi|2296125236|ref|NG_008477.2|
Description: Homo sapiens SURF1 cytochrome c oxidase assembly factor (SURF1), RefSeqGene on chromosome 9
E-value: 2.19268e-13
Score: 105.0
----------------------------------------
Hit ID: gi|242380115|emb|AL593848.15|
Description: Human DNA sequence from clone RP13-100B2 on chromosome 9, complete sequence
E-value: 0.0
Score: 23460.0
----------------------------------------
Hit ID: gi|242380115|emb|AL593848.15|
Description: Human DNA sequence from clone RP13-100B2 on chromosome 9, compl

In [12]:
import pandas as pd

# Convert parsed results to a DataFrame
df = pd.DataFrame(parsed_results)

# Save to CSV
df.to_csv("parsed_blast_results.csv", index=False)
print("Parsed BLAST results saved to 'parsed_blast_results.csv'")

Parsed BLAST results saved to 'parsed_blast_results.csv'


## Result Interpretaion 

Exact Match: The first hit has an E-value of 0.0 and a very high score of 23,460.0, which tells us that the query sequence NG_008477.2 has been matched almost perfectly to itself or an identical sequence in the database. This is expected when the exact same sequence is already present in the database.

Additional Matches: The next two hits have slightly lower scores (105.0) and E-values of 2.19e-13. These are still extremely good matches, showing that these sequences are nearly identical to the query but may represent slightly shorter or overlapping parts of the SURF1 gene. Since these hits are from the same RefSeqGene entry as the first hit, it further confirms they are closely related or identical to our query sequence, with some minor differences.